# 환경구축

데이터 파일들을 불러오고 저장하기 위해서 Google Drive를 연결합니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.bashrc.biolab
./root/.bin.priority/
./root/.bin.priority/pip3
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.tmux.conf
./root/.vimrc
./root/.profile
./root/.condarc
--2022-06-09 17:46:31--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M   177MB/s    in 0.4s    

2022-06-09 17:46:32 (177 MB/s) - ‘miniconda3.sh’ save

In [3]:
!md5sum drive/MyDrive/binfo1-datapack1/*

140aaf30bcb9276cc716f8699f04ddd6  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam.bai
9f43477870337dfddbc154809a2a3ec8  drive/MyDrive/binfo1-datapack1/CLIP-let7d.bam
8f0013aa795510f597e241c1b69e6341  drive/MyDrive/binfo1-datapack1/CLIP-let7d-gene.pileup
8f0013aa795510f597e241c1b69e6341  drive/MyDrive/binfo1-datapack1/CLIP-let7d.pileup
d0bc27a4926b599fb905ce2c530f259b  drive/MyDrive/binfo1-datapack1/CLIP-let7d_sorted.bam
d68c64201fb0c3c301fbb1efdde8311d  drive/MyDrive/binfo1-datapack1/CLIP-let7d_sorted.bam.bai
d5d8996d383d994fe1916bf239f91964  drive/MyDrive/binfo1-datapack1/CLIP-let7f-1.bam
1c34f27bcc5f407e54d8112938bc9d2c  drive/MyDrive/binfo1-datapack1/CLIP-let7f-1-gene.pileup
9b172b22ce452b8f7ef1b02dcf61516b  drive/MyDrive/binfo1-datapack1/CLIP-let7f-1.pileup
0d33bf4b3a5ce22ba13a62323efd6883  drive/MyDrive/binfo1-datapack1/CLIP-let7g.bam
4ea14471c5d4bf9ab1fd91b9a5b2193c  drive/MyDrive/binfo1-datapack1/CLI

실습에 필요한 프로그램들 몇 가지를 추가로 설치합니다.

In [4]:
!conda install -y bedtools bioawk

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /root/conda/envs/lab

  added / updated specs:
    - bedtools
    - bioawk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bioawk-1.0                 |       h7132678_7         196 KB  bioconda
    ------------------------------------------------------------
                                           Total:         196 KB

The following NEW packages will be INSTALLED:

  bioawk             bioconda/linux-64::bioawk-1.0-h7132678_7



bioawk-1.0           | 196 KB    | : 100% 1.0/1 [00:00<00:00,  9.93it/s]
Preparing transaction: / done
Ve

# Fig S2A 재현해보기
error가 많이 나오는 부분의 unique sequence들의 각각 개수를 세서 genome browser로 표현하고, 점수 트랙을 같이 표시

In [5]:
%cd /content/drive/MyDrive/binfo1-datapack1/

/content/drive/MyDrive/binfo1-datapack1


논문은 예전 레퍼런스 유전체인 mm9을 쓰고 있기 때문에, 현재 사용되는 레퍼런스 mm39와는 좌표계에 차이가 있습니다. 새로 annotation에서 위치를 찾습니다.

In [6]:
!grep -i mirlet7f-1 gencode.gtf

chr13	ENSEMBL	gene	48691305	48691393	.	-	.	gene_id "ENSMUSG00000105621.3"; gene_type "miRNA"; gene_name "Mirlet7f-1"; level 3; mgi_id "MGI:2676798";
chr13	ENSEMBL	transcript	48691305	48691393	.	-	.	gene_id "ENSMUSG00000105621.3"; transcript_id "ENSMUST00000198652.3"; gene_type "miRNA"; gene_name "Mirlet7f-1"; transcript_type "miRNA"; transcript_name "Mirlet7f-1-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676798"; tag "basic";
chr13	ENSEMBL	exon	48691305	48691393	.	-	.	gene_id "ENSMUSG00000105621.3"; transcript_id "ENSMUST00000198652.3"; gene_type "miRNA"; gene_name "Mirlet7f-1"; transcript_type "miRNA"; transcript_name "Mirlet7f-1-201"; exon_number 1; exon_id "ENSMUSE00001348852.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676798"; tag "basic";


Splicing 되지 않는 transcript이기 때문에 결과가 단조롭습니다. 해당 좌표 범위로 bam을 솎아냅니다.

In [8]:
!samtools view -b -o CLIP-let7f-1.bam CLIP-35L33G.bam chr13: 48691305-48691393
!samtools view CLIP-let7f-1.bam | wc -l

[main_samview] region "48691305-48691393" specifies an invalid region or unknown reference. Continue anyway.
1669714


samtools를 이용해 bam file을 sort하고 index file (bam.bai) 생성

In [9]:
!samtools sort CLIP-let7f-1.bam -o CLIP-let7f-1_sorted.bam

In [10]:
!samtools index CLIP-let7f-1_sorted.bam

각 자리에 mapping된 read들의 분포를 요약하기 위해 samtools mpileup 이용하기.

In [11]:
!samtools mpileup CLIP-let7f-1.bam > CLIP-let7f-1.pileup
!wc -l CLIP-let7f-1.pileup

[mpileup] 1 samples in 1 input files
44620190 CLIP-let7f-1.pileup


Mirlet7f-1의 핵심부분을 살펴봐서 어떻게 하면 좋을지 견적을 내보기.

In [12]:
!head CLIP-let7f-1.pileup

chr13	3053463	N	1	^IA	I
chr13	3053464	N	1	G	I
chr13	3053465	N	1	A	I
chr13	3053466	N	1	A	I
chr13	3053467	N	1	A	I
chr13	3053468	N	1	T	I
chr13	3053469	N	1	G	I
chr13	3053470	N	1	G	I
chr13	3053471	N	1	G	I
chr13	3053472	N	1	C	I


In [13]:
!awk '$2 >= 48691305 && $2 <= 48691393 { print $0; }' CLIP-let7f-1.pileup > CLIP-let7f-1-gene.pileup
!tail CLIP-let7f-1-gene.pileup

chr13	48691377	N	24	tttttttttttttttttttttttt	BIIGIGEDCGFIFGI@GHIDDIGI
chr13	48691378	N	24	aaaaaaaaaaaaaaaaaaaaaaaa	?HIFIGGGAFEFABI>GII;EIGI
chr13	48691379	N	24	cccccccccccccccccccccccc	EIIGIGGDEGGIEGI;GIIDDIGI
chr13	48691380	N	24	tttttttttttttttttttttttt	EIIGIBGGEGFIFGI@GIIBDIGI
chr13	48691381	N	24	aaaaaaaaaaaaaaaaaaaaaaaa	BEHFIGEG@FCEAGH4GII;AIDH
chr13	48691382	N	24	cccccccccccccccccccccccc	?IEGIHGG?GEGEBI=GIIDEIGG
chr13	48691383	N	24	cccccccccccccccccccccccc	EIGGIGGGAGFGFGI=GIIDGIGG
chr13	48691384	N	24	tttttttttttttttttttttttt	<IGGIHGG>GEGBGI8GII@GIGG
chr13	48691385	N	24	cccccccccccccccccccccccc	BIGGIBDD?GEGBEI=GIIBGIGG
chr13	48691386	N	24	a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$	EIGGIFDECGFGFGI2BIIDGIGG


In [14]:
!ls

CLIP-35L33G.bam			       gencode-exons-containing-startcodon.bed
CLIP-35L33G.bam.bai		       gencode.gtf
CLIP-let7d.bam			       gencode-plusexon.gtf
CLIP-let7d-gene.pileup		       gencode-start.gtf
CLIP-let7d.pileup		       let7d.bed
CLIP-let7d_sorted.bam		       Mirlet7d.txt
CLIP-let7d_sorted.bam.bai	       Mirlet7f-1.txt
CLIP-let7f-1.bam		       Mirlet7g.txt
CLIP-let7f-1-gene.pileup	       read-counts.txt
CLIP-let7f-1.pileup		       read-counts.txt.summary
CLIP-let7f-1_sorted.bam		       RNA-control.bam
CLIP-let7f-1_sorted.bam.bai	       RNA-control.bam.bai
CLIP-let7g.bam			       RNA-siLin28a.bam
CLIP-let7g-gene.pileup		       RNA-siLin28a.bam.bai
CLIP-let7g.pileup		       RNA-siLuc.bam
filtered-RPF-siLin28a.bam	       RNA-siLuc.bam.bai
filtered-RPF-siLuc.bam		       RPF-siLin28a.bam
fivepcounts-filtered-RPF-siLin28a.txt  RPF-siLin28a.bam.bai
fivepcounts-filtered-RPF-siLuc.txt     RPF-siLuc.bam
fivepcounts-RPF-siLin28a.bed	       RPF-siLuc.bam.bai
fivepcounts-RPF-siLuc.bed


이제 pandas에 불러와서 분석하기.

In [15]:
import pandas as pd
dpileup = pd.read_csv('CLIP-let7f-1-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
dpileup.tail()

,chrom,pos,_ref,count,basereads,quals
77,chr13,48691382,N,24,cccccccccccccccccccccccc,?IEGIHGG?GEGEBI=GIIDEIGG
78,chr13,48691383,N,24,cccccccccccccccccccccccc,EIGGIGGGAGFGFGI=GIIDGIGG
79,chr13,48691384,N,24,tttttttttttttttttttttttt,<IGGIHGG>GEGBGI8GII@GIGG
80,chr13,48691385,N,24,cccccccccccccccccccccccc,BIGGIBDD?GEGBEI=GIIBGIGG
81,chr13,48691386,N,24,a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$a$,EIGGIFDECGFGFGI2BIIDGIGG


여기서 pileup에서 계산에 사용할 것은 match와 substitution이기 때문에 나머지에 해당하는 tag들 제거하기.

In [16]:
import re
toremove = re.compile('[<>$*#^]')
dpileup['matches'] = dpileup['basereads'].apply(lambda x: toremove.sub('', x))

In [17]:
dpileup[['chrom', 'pos', 'matches']]

,chrom,pos,matches
0,chr13,48691305,cccccccccccccccccccccccccccccccccccccccccccccc...
1,chr13,48691306,tttttttttttttttttttttttttttttttttttttttttttttt...
2,chr13,48691307,cccccccccccccccccccccccccccccccccccccccccccccc...
3,chr13,48691308,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
4,chr13,48691309,gggggggggggggggggggggggggggggggggggggggggggggg...
...,...,...,...
77,chr13,48691382,cccccccccccccccccccccccc
78,chr13,48691383,cccccccccccccccccccccccc
79,chr13,48691384,tttttttttttttttttttttttt
80,chr13,48691385,cccccccccccccccccccccccc


In [18]:
dpileup[dpileup['pos'] == 48691309].iloc[0]['matches']

'ggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggg'

In [19]:
dpileup2 = dpileup[['chrom', 'pos', 'matches']]

In [20]:
dpileup3 = dpileup2['matches']

In [21]:
dpileup2['base'] = dpileup3.str.len()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
dpileup2

,chrom,pos,matches,base
0,chr13,48691305,cccccccccccccccccccccccccccccccccccccccccccccc...,109
1,chr13,48691306,tttttttttttttttttttttttttttttttttttttttttttttt...,109
2,chr13,48691307,cccccccccccccccccccccccccccccccccccccccccccccc...,109
3,chr13,48691308,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,109
4,chr13,48691309,gggggggggggggggggggggggggggggggggggggggggggggg...,109
...,...,...,...,...
77,chr13,48691382,cccccccccccccccccccccccc,24
78,chr13,48691383,cccccccccccccccccccccccc,24
79,chr13,48691384,tttttttttttttttttttttttt,24
80,chr13,48691385,cccccccccccccccccccccccc,24


In [23]:
import math
from collections import Counter

def entropy(s):
	p, lns = Counter(s), float(len(s))
	return -sum( count/lns * math.log(count/lns, 2) for count in p.values())

In [24]:
count = ['1','2']
dent = pd.Series(count)
print(dent)
print(dent[1])

0    1
1    2
dtype: object
2


In [25]:
count = ['1','2']
ent = pd.Series(count)
print(ent)
print(ent[1])

0    1
1    2
dtype: object
2


In [27]:
i = 0
while i <= 81:
  dent[i] = entropy(dpileup3[i])
  i += 1

In [28]:
dpileup4 = dpileup2
dpileup4['Shannon_entropy'] = dent
print(dpileup4)

    chrom       pos                                            matches  base  \
0   chr13  48691305  cccccccccccccccccccccccccccccccccccccccccccccc...   109   
1   chr13  48691306  tttttttttttttttttttttttttttttttttttttttttttttt...   109   
2   chr13  48691307  cccccccccccccccccccccccccccccccccccccccccccccc...   109   
3   chr13  48691308  aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...   109   
4   chr13  48691309  gggggggggggggggggggggggggggggggggggggggggggggg...   109   
..    ...       ...                                                ...   ...   
77  chr13  48691382                           cccccccccccccccccccccccc    24   
78  chr13  48691383                           cccccccccccccccccccccccc    24   
79  chr13  48691384                           tttttttttttttttttttttttt    24   
80  chr13  48691385                           cccccccccccccccccccccccc    24   
81  chr13  48691386                           aaaaaaaaaaaaaaaaaaaaaaaa    24   

   Shannon_entropy  
0             -0.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
dpileup5 = dpileup4[['chrom','pos']]
dpileup5['pos2']=dpileup4['pos']+1
dpileup5['Shannon_entropy'] = dpileup4['Shannon_entropy']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
dpileup5

,chrom,pos,pos2,Shannon_entropy
0,chr13,48691305,48691306,-0.0
1,chr13,48691306,48691307,-0.0
2,chr13,48691307,48691308,-0.0
3,chr13,48691308,48691309,-0.0
4,chr13,48691309,48691310,-0.0
...,...,...,...,...
77,chr13,48691382,48691383,-0.0
78,chr13,48691383,48691384,-0.0
79,chr13,48691384,48691385,-0.0
80,chr13,48691385,48691386,-0.0


In [31]:
dpileup5.to_csv("Mirlet7f-1.txt",header=False, index=False, sep='\t')

In [32]:
dpileup5sorted = dpileup5

In [33]:
dpileup5sorted.sort_values('Shannon_entropy',ascending=False)

,chrom,pos,pos2,Shannon_entropy
33,chr13,48691338,48691339,1.273183
16,chr13,48691321,48691322,0.796347
7,chr13,48691312,48691313,0.657262
32,chr13,48691337,48691338,0.619382
5,chr13,48691310,48691311,0.39751
...,...,...,...,...
27,chr13,48691332,48691333,-0.0
26,chr13,48691331,48691332,-0.0
25,chr13,48691330,48691331,-0.0
24,chr13,48691329,48691330,-0.0


In [34]:
dpileup5.to_csv("let7f-1.bed", sep="\t", header=None, index=False)